In [6]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/")

In [7]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://ms-mlflow-bucket/500246362129033379', creation_time=1751027568684, experiment_id='500246362129033379', last_update_time=1751027568684, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna


In [9]:
df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

(36243, 2)

In [10]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic Regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 2000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2025-06-27 21:08:56,448] A new study created in memory with name: no-name-689e96a6-bea2-4a2a-9f92-b5b741db81e9
[I 2025-06-27 21:08:56,674] Trial 0 finished with value: 0.3332978723404255 and parameters: {'C': 0.00017455617520779307, 'penalty': 'l1'}. Best is trial 0 with value: 0.3332978723404255.
[I 2025-06-27 21:08:57,022] Trial 1 finished with value: 0.6385106382978724 and parameters: {'C': 0.0003054025245237227, 'penalty': 'l2'}. Best is trial 1 with value: 0.6385106382978724.
[I 2025-06-27 21:08:57,553] Trial 2 finished with value: 0.6437234042553192 and parameters: {'C': 0.002073019877187721, 'penalty': 'l2'}. Best is trial 2 with value: 0.6437234042553192.
[I 2025-06-27 21:08:57,947] Trial 3 finished with value: 0.6420212765957447 and parameters: {'C': 0.0013926633467337794, 'penalty': 'l2'}. Best is trial 2 with value: 0.6437234042553192.
[I 2025-06-27 21:09:00,391] Trial 4 finished with value: 0.8057446808510639 and parameters: {'C': 3.1669826616417143, 'penalty': 'l2'}. Be

🏃 View run LogisticRegression_SMOTE_TFIDF_Trigrams at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/500246362129033379/runs/7985d14c180f4f66b2be1feddb6b8c21
🧪 View experiment at: http://ec2-18-233-10-235.compute-1.amazonaws.com:5000/#/experiments/500246362129033379
